1) 파일 변환 작업 
    xlsx -> csv 

In [5]:
# Step 1: Import packages
import pandas as pd
import re
import os
import json
from datetime import datetime

In [ ]:
df = pd.read_excel("korea_rental_test.xlsx")
df.to_csv("한국렌탈_렌탈료.csv", index=False, encoding='utf-8')

In [3]:
# 파일 경로 설정
input_file = '한국렌탈_렌탈료.csv'
output_file = '자동전표_업로드.csv'
team_mapping_file = 'team_name_mapping.json'

In [78]:
# # Step 3: Load 매핑 JSON 파일
# with open("team_name_mapping.json", encoding="utf-8") as f:
#     raw_json = json.load(f)

# # 'present' → 'past' 구조로 딕셔너리 변환
# team_map = {entry["present"]: entry["past"] for entry in raw_json["parameterMapping"]}

# print("매핑 JSON 변환 결과 (예시 10개)")
# print(dict(list(team_map.items())[:10]))

# 팀명 매핑 정보 로드
print(f"'{team_mapping_file}' 파일에서 팀명 매핑 정보 로드 중...")
with open(team_mapping_file, 'r', encoding='utf-8') as f:
    team_name_mapping = json.load(f)

# 부서/팀 매핑 정보 (실제 데이터로 업데이트)
team_map = team_name_mapping

# CSV 파일 로드
print(f"'{input_file}' 파일 로딩 중...")
rental_df = pd.read_csv(input_file, encoding='utf-8')
print(f"로딩 완료: {len(rental_df)}개 행 발견")

'team_name_mapping.json' 파일에서 팀명 매핑 정보 로드 중...
'한국렌탈_렌탈료.csv' 파일 로딩 중...
로딩 완료: 4920개 행 발견


### 테스트용 일부 데이터 추출 코드

In [14]:
import pandas as pd
import re
import os
import json
from datetime import datetime

# 파일 경로 설정 (ERP 양식 파일 추가)
input_file = '한국렌탈_렌탈료.csv'
output_csv = '(테스트)자동전표_업로드.csv'
output_excel = '(테스트)자동전표_업로드.xlsx'
mapping_file = 'test.json'
erp_form_file = 'erp_form.csv'

# 매핑 정보 로드
print(f"'{mapping_file}' 파일에서 매핑 정보 로드 중...")
with open(mapping_file, 'r', encoding='utf-8') as f:
    mapping_list = json.load(f)

# 매핑 딕셔너리 생성
mapping_dict = {}
for item in mapping_list:
    mapping_dict[item['past']] = {
        'present': item['present'],
        'CD_ACCT': item['CD_ACCT'],
        'CD_PJT': item['CD_PJT']
    }

print(f"매핑 정보 로드 완료: {len(mapping_dict)}개 항목")

# CSV 파일 로드
print(f"'{input_file}' 파일 로딩 중...")
rental_df = pd.read_csv(input_file, encoding='utf-8')
print(f"로딩 완료: {len(rental_df)}개 행 발견")

# 전처리 과정
print("데이터 전처리 중...")
df = rental_df[["모델명", "3월렌탈료", "영업분류", "관리부서", "거래처명", "관리지점", "2월 PJT", "3월 변경PJT"]].copy()

# 금액 필드 변환 (정수 타입으로)
df["금액"] = df["3월렌탈료"].replace(",", "", regex=True).astype(float).astype(int)

# 팀명 처리 (3월 변경PJT 우선, 없으면 2월 PJT 사용)
df["원본팀명"] = df["3월 변경PJT"].combine_first(df["2월 PJT"])

# 매핑 적용 함수
def apply_mapping(team_name):
    if pd.isna(team_name) or team_name == "":
        return {"present": "", "CD_ACCT": "", "CD_PJT": ""}
    
    if team_name in mapping_dict:
        return mapping_dict[team_name]
    
    # 없는 경우 빈 값 반환
    return {"present": team_name, "CD_ACCT": "", "CD_PJT": ""}

# 매핑 적용
df["매핑정보"] = df["원본팀명"].apply(apply_mapping)

# 매핑 정보에서 필드 추출
df["팀명"] = df["매핑정보"].apply(lambda x: x["present"])
df["CD_ACCT"] = df["매핑정보"].apply(lambda x: x["CD_ACCT"])
df["CD_PJT"] = df["매핑정보"].apply(lambda x: x["CD_PJT"])

# 적요 생성
df["적요"] = "테스트 한국렌탈㈜_PC 렌탈료(" + df["팀명"] + ")"

# MNG 코드 설정
df["CD_MNG1"] = "5020"  # 코스트센터(운영2)
df["CD_MNG3"] = "101388"  # 거래처 코드

# 필요한 관리항목 코드값만 설정. 관리내역은 ERP 시스템에서 자동으로 생성될 것임

# 매핑된 항목만 선택 (CD_ACCT와 CD_PJT가 있는 항목만)
df_filtered = df[(df["CD_ACCT"] != "") & (df["CD_PJT"] != "")].copy()
print(f"매핑된 항목: {len(df_filtered)}개 / 전체 {len(df)}개")

# 금액 확인
print("금액 필드 확인:")
print(df_filtered["금액"].head())

# ERP 업로드용 데이터프레임 생성
print("ERP 업로드용 데이터프레임 생성 중...")
current_date = datetime.now().strftime("%Y%m%d")
document_number = f"AUTO{current_date[-4:]}"  # AUTO + 날짜 끝 4자리

# 차변 데이터와 대변 데이터를 별도로 생성 후 합칠 예정
# 1. 차변 데이터 생성 (각 팀별 계정별 비용)
debit_data = {
    "ROW_ID": ["FI2025042400005"] * len(df_filtered), 
    "ROW_NO": [str(i) for i in range(1, len(df_filtered)+1)],
    "NO_TAX": ["*"] * len(df_filtered),
    "CD_PC": ["1200"] * len(df_filtered),
    "CD_WDEPT": ["1010"] * len(df_filtered),
    "NO_DOCU": ["FI2025042400005"] * len(df_filtered), 
    "NO_DOLINE": [str(i) for i in range(1, len(df_filtered)+1)],
    "CD_COMPANY": ["1200"] * len(df_filtered),
    "ID_WRITE": ["00616"] * len(df_filtered), # 사원번호 김하리
    "CD_DOCU": ["11"] * len(df_filtered), # 전표유형 (11: 일반) 고정
    "DT_ACCT": [current_date] * len(df_filtered),
    "ST_DOCU": ["1"] * len(df_filtered), # 승인여부 (1: 미결/임시)
    "TP_DRCR": ["1"] * len(df_filtered), # 차대구분 (1: 차변)
    "CD_ACCT": df_filtered["CD_ACCT"].tolist(), # 각 팀별 계정 코드
    "AMT": df_filtered["금액"].astype(str).tolist(), 
    "CD_PARTNER": ["101388"] * len(df_filtered), # 거래처코드 (한국렌탈: 101388) 고정
    "NM_NOTE": df_filtered["적요"].tolist(),
    "TP_DOCU": ["N"] * len(df_filtered),    # 전표처리결과 (N: 미처리/임시)
    "NO_ACCT": ["0"] * len(df_filtered),    # 회계승인번호 (0 고정)
    "TP_GUBUN": ["3"] * len(df_filtered),    # 전표구분 (3: 대체전표) 고정
}

# 2. 대변 데이터 생성 (미지급금으로 합계 금액)
# 총 금액 계산
total_amount = df_filtered["금액"].sum()
total_amount_str = str(total_amount)

# 대변 데이터
credit_data = {
    "ROW_ID": ["FI2025042400005"], 
    "ROW_NO": [str(len(df_filtered) + 1)],  # 마지막 번호 다음
    "NO_TAX": ["*"],
    "CD_PC": ["1200"],
    "CD_WDEPT": ["1010"],
    "NO_DOCU": ["FI2025042400005"], 
    "NO_DOLINE": [str(len(df_filtered) + 1)],  # 마지막 라인 다음
    "CD_COMPANY": ["1200"],
    "ID_WRITE": ["00616"], # 사원번호 김하리
    "CD_DOCU": ["11"], # 전표유형 (11: 일반) 고정
    "DT_ACCT": [current_date],
    "ST_DOCU": ["1"], # 승인여부 (1: 미결/임시)
    "TP_DRCR": ["2"], # 차대구분 (2: 대변)
    "CD_ACCT": ["25300"], # 미지급금 계정코드
    "AMT": [total_amount_str],  # 전체 금액의 합계
    "CD_PARTNER": ["101388"], # 거래처코드 (한국렌탈: 101388) 고정
    "NM_NOTE": ["테스트 한국렌탈㈜_PC 렌탈료 미지급금"],  # 적요
    "TP_DOCU": ["N"],    # 전표처리결과 (N: 미처리/임시)
    "NO_ACCT": ["0"],    # 회계승인번호 (0 고정)
    "TP_GUBUN": ["3"],    # 전표구분 (3: 대체전표) 고정
}

# 3. 차변과 대변 데이터프레임 생성
debit_df = pd.DataFrame(debit_data)
credit_df = pd.DataFrame(credit_data)

# 4. 두 데이터프레임 합치기
erp_df = pd.concat([debit_df, credit_df], ignore_index=True)

# 금액 필드 확인
print("\nAMT 필드 확인:")
print("차변 금액 합계:", df_filtered["금액"].sum())
print("대변 금액:", total_amount)
print("차변 건수:", len(debit_df))
print("대변 건수:", len(credit_df))

# 필드 순서 지정 - ERP 양식에 맞게 정확한 순서로 컬럼 정렬
erp_columns = [
    "ROW_ID", "ROW_NO", "NO_TAX", "CD_PC", "CD_WDEPT", "NO_DOCU", "NO_DOLINE", 
    "CD_COMPANY", "ID_WRITE", "CD_DOCU", "DT_ACCT", "ST_DOCU", "TP_DRCR", 
    "CD_ACCT", "AMT", "CD_PARTNER", "DT_START", "DT_END", "AM_TAXSTD", 
    "AM_ADDTAX", "TP_TAX", "NO_COMPANY", "NM_NOTE", "CD_BIZAREA", "CD_DEPT", 
    "CD_CC", "CD_PJT", "CD_FUND", "CD_BUDGET", "NO_CASH", "ST_MUTUAL", 
    "CD_CARD", "NO_DEPOSIT", "CD_BANK", "UCD_MNG1", "UCD_MNG2", "UCD_MNG3", 
    "UCD_MNG4", "UCD_MNG5", "CD_EMPLOY", "CD_MNG", "NO_BDOCU", "NO_BDOLINE", 
    "TP_DOCU", "NO_ACCT", "TP_TRADE", "NO_CHECK3", "NO_CHECK4", "CD_EXCH", 
    "RT_EXCH", "CD_TRADE", "AM_EX", "TP_EXPORT", "NO_TO", "DT_SHIPPING", 
    "TP_GUBUN", "NO_INVOICE", "NO_ITEM", "MD_TAX1", "NM_ITEM1", "NM_SIZE1", 
    "QT_TAX1", "AM_PRC1", "AM_SUPPLY1", "AM_TAX1", "NM_NOTE1", "CD_BIZPLAN", 
    "CD_BGACCT", "CD_MNGD1", "NM_MNGD1", "CD_MNGD2", "NM_MNGD2", "CD_MNGD3", 
    "NM_MNGD3", "CD_MNGD4", "NM_MNGD4", "CD_MNGD5", "NM_MNGD5", "CD_MNGD6", 
    "NM_MNGD6", "CD_MNGD7", "NM_MNGD7", "CD_MNGD8", "NM_MNGD8", "YN_ISS", 
    "FINAL_STATUS", "NO_BILL", "NM_BIGO", "TP_BILL", "TP_RECORD", "TP_ETCACCT", 
    "ST_GWARE", "SELL_DAM_NM", "SELL_DAM_EMAIL", "SELL_DAM_MOBIL", "SELL_DAM_TEL", 
    "NM_PUMM", "JEONJASEND15_YN", "DT_WRITE", "ST_TAX", "MD_TAX2", "NM_ITEM2", 
    "NM_SIZE2", "QT_TAX2", "AM_PRC2", "AM_SUPPLY2", "AM_TAX2", "NM_NOTE2", 
    "MD_TAX3", "NM_ITEM3", "NM_SIZE3", "QT_TAX3", "AM_PRC3", "AM_SUPPLY3", 
    "AM_TAX3", "NM_NOTE3", "MD_TAX4", "NM_ITEM4", "NM_SIZE4", "QT_TAX4", 
    "AM_PRC4", "AM_SUPPLY4", "AM_TAX4", "NM_NOTE4", "NM_PTR", "EX_HP", 
    "EX_EMIL", "NO_BIZTAX", "NO_ASSET", "TP_EVIDENCE", "NO_CAR", "NO_CARBODY", 
    "CD_BIZCAR", "NM_PARTNER", "YN_IMPORT", "YN_FIXASSET"
]

# 나머지 열 추가 (빈 문자열로)
for col in erp_columns:
    if col not in erp_df.columns:
        erp_df[col] = [""] * len(erp_df)

# CD_CC 및 CD_PJT 필드 설정 (올바른 위치에)
# 차변 행 설정
debit_rows = erp_df["TP_DRCR"] == "1"
erp_df.loc[debit_rows, "CD_CC"] = "5020"  # 코스트센터
erp_df.loc[debit_rows, "CD_PJT"] = df_filtered["CD_PJT"].tolist()  # 프로젝트 코드

# 대변 행 설정
credit_rows = erp_df["TP_DRCR"] == "2"
erp_df.loc[credit_rows, "CD_CC"] = "5020"  # 코스트센터
# 대변에는 프로젝트 코드 필요 없음

# 최종 데이터프레임을 정확한 순서로 재구성
erp_df = erp_df[erp_columns]

# 필드 확인
print("\n필드 순서 및 값 확인:")
print(f"- CD_CC 위치: {erp_columns.index('CD_CC')}번째 컬럼")
print(f"- CD_PJT 위치: {erp_columns.index('CD_PJT')}번째 컬럼")
print(f"- 미지급금(25300) 확인: {sum(erp_df['CD_ACCT'] == '25300')}개 행 발견")
print(f"- 차대구분(TP_DRCR) 확인: 차변(1) {sum(erp_df['TP_DRCR'] == '1')}건, 대변(2) {sum(erp_df['TP_DRCR'] == '2')}건")

# CSV 및 엑셀 파일 저장 부분 수정
print(f"'{output_csv}'로 CSV 저장 중...")

# ERP 양식 파일 로드 (첫 4행은 유지)
try:
    erp_form = pd.read_csv(erp_form_file, encoding='utf-8')
    has_form_template = True
    print("ERP 양식 파일을 성공적으로 로드했습니다.")
except Exception as e:
    has_form_template = False
    print(f"ERP 양식 파일 로드 실패: {e}")
    print("기본 양식 없이 진행합니다.")

if has_form_template:
    # 양식 파일의 컬럼 순서 사용
    form_columns = erp_form.columns.tolist()
    
    # 결과 데이터프레임을 양식 컬럼 순서에 맞게 재정렬
    for col in form_columns:
        if col not in erp_df.columns:
            erp_df[col] = ""
    
    erp_df = erp_df[form_columns]
    
    # erp_form 복사
    result_df = erp_form.copy()
    
    # 빈 행 1개 추가 (4행)
    result_df = pd.concat([result_df, pd.DataFrame([[""] * len(form_columns)], columns=form_columns)], ignore_index=True)
    
    # 처리된 데이터 추가 (5행부터 시작)
    result_df = pd.concat([result_df, erp_df], ignore_index=True)
    
    # 결과 저장
    result_df.to_csv(output_csv, index=False, encoding='utf-8-sig')
    print(f"처리 완료: {len(erp_df)}개 행이 '{output_csv}'에 저장됨 (UTF-8-SIG 인코딩)")
    print(f"양식 파일의 첫 4행이 유지되었습니다. 데이터는 5행부터 시작합니다.")
    
    # 엑셀 파일 저장
    print(f"\n'{output_excel}'로 엑셀 파일 저장 중...")
    try:
        result_df.to_excel(output_excel, index=False, engine='openpyxl')
        print(f"처리 완료: {len(erp_df)}개 행이 '{output_excel}'에 저장됨")
        print(f"엑셀 파일이 성공적으로 생성되었습니다: {os.path.abspath(output_excel)}")
    except Exception as e:
        print(f"엑셀 파일 저장 중 오류 발생: {e}")
        print("CSV 파일은 정상적으로 저장되었습니다.")
else:
    # 기존 방식대로 저장 (양식 파일이 없는 경우)
    erp_df.to_csv(output_csv, index=False, encoding='utf-8-sig')
    print(f"처리 완료: {len(erp_df)}개 행이 '{output_csv}'에 저장됨 (UTF-8-SIG 인코딩)")
    print(f"주의: ERP 양식 파일을 찾지 못해 기본 형식으로 저장되었습니다. 데이터가 5행부터 시작하도록 수동으로 조정해야 합니다.")
    
    # 엑셀 파일 저장
    print(f"\n'{output_excel}'로 엑셀 파일 저장 중...")
    try:
        erp_df.to_excel(output_excel, index=False, engine='openpyxl')
        print(f"처리 완료: {len(erp_df)}개 행이 '{output_excel}'에 저장됨")
        print(f"엑셀 파일이 성공적으로 생성되었습니다: {os.path.abspath(output_excel)}")
    except Exception as e:
        print(f"엑셀 파일 저장 중 오류 발생: {e}")
        print("CSV 파일은 정상적으로 저장되었습니다.")
        print("CSV 파일을 열 때는 Excel의 '데이터' 탭에서 '텍스트/CSV에서' 기능을 사용하시기 바랍니다.")

# 데이터 요약 정보
total_amount = df_filtered["금액"].sum()
print(f"\n총 처리 건수: {len(df_filtered) + 1}건 (차변 {len(df_filtered)}건, 대변 1건)")
print(f"총 금액: {total_amount:,.0f}원")
print(f"차변 계정: {len(df_filtered['CD_ACCT'].unique())}개 계정 사용")
print(f"대변 계정: 미지급금(25300) 1개 계정 사용")

# 관리항목 설정 내용 출력
print("\n관리항목 설정 정보:")
print(f"- CD_CC (코스트센터): 5020 (고정)")
print(f"- CD_PARTNER (거래처코드): 101388 (고정)")
print(f"- CD_PJT (프로젝트코드): 각 팀별 매핑된 코드 사용")

# 매핑 정보 요약
print("\n매핑 성공 팀명:")
mapped_teams = df_filtered["원본팀명"].unique()
for team in mapped_teams[:10]:  # 처음 10개만 표시
    if len(mapped_teams) > 10 and team == mapped_teams[9]:
        print(f"- {team} ... 외 {len(mapped_teams)-10}개")
    else:
        mapped_info = mapping_dict.get(team, {})
        print(f"- {team} -> {mapped_info.get('present', team)} (ACCT: {mapped_info.get('CD_ACCT', '')}, PJT: {mapped_info.get('CD_PJT', '')})")

# 매핑 실패 팀명 (필터링된 항목)
unmapped_df = df[(df["CD_ACCT"] == "") | (df["CD_PJT"] == "")]
unmapped_teams = unmapped_df["원본팀명"].unique()
print(f"\n매핑 실패 팀명: {len(unmapped_teams)}개")
for team in unmapped_teams[:10]:  # 처음 10개만 표시
    if len(unmapped_teams) > 10 and team == unmapped_teams[9]:
        print(f"- {team} ... 외 {len(unmapped_teams)-10}개")
    else:
        print(f"- {team}")

# 필드 설정 요약
print("\n설정된 주요 필드:")
print(f"- ROW_ID 예시: {erp_df['ROW_ID'].iloc[0]} (자동생성 예정)")
print(f"- NO_DOCU: {erp_df['NO_DOCU'].iloc[0]} (자동생성 예정)")
print(f"- DT_ACCT: {erp_df['DT_ACCT'].iloc[0]}")
print(f"- ST_DOCU: {erp_df['ST_DOCU'].iloc[0]} (미결)")
print(f"- CD_PARTNER: {erp_df['CD_PARTNER'].iloc[0]}")
print(f"- 차변 계정코드 예시: {erp_df['CD_ACCT'].iloc[0]}")
print(f"- 대변 계정코드: 25300 (미지급금)")
print(f"- 차변 AMT 합계: {df_filtered['금액'].sum():,.0f}원")
print(f"- 대변 AMT 합계: {total_amount:,.0f}원")
print(f"- 차대구분(TP_DRCR): 차변(1) {sum(erp_df['TP_DRCR'] == '1')}건, 대변(2) {sum(erp_df['TP_DRCR'] == '2')}건")

'test.json' 파일에서 매핑 정보 로드 중...
매핑 정보 로드 완료: 7개 항목
'한국렌탈_렌탈료.csv' 파일 로딩 중...
로딩 완료: 4920개 행 발견
데이터 전처리 중...
매핑된 항목: 59개 / 전체 4920개
금액 필드 확인:
21     1700
68     2340
115    7050
250    3375
252    6525
Name: 금액, dtype: int64
ERP 업로드용 데이터프레임 생성 중...

AMT 필드 확인:
차변 금액 합계: 259080
대변 금액: 259080
차변 건수: 59
대변 건수: 1

필드 순서 및 값 확인:
- CD_CC 위치: 25번째 컬럼
- CD_PJT 위치: 26번째 컬럼
- 미지급금(25300) 확인: 1개 행 발견
- 차대구분(TP_DRCR) 확인: 차변(1) 59건, 대변(2) 1건
'(테스트)자동전표_업로드.csv'로 CSV 저장 중...
ERP 양식 파일을 성공적으로 로드했습니다.
처리 완료: 60개 행이 '(테스트)자동전표_업로드.csv'에 저장됨 (UTF-8-SIG 인코딩)
양식 파일의 첫 4행이 유지되었습니다. 데이터는 5행부터 시작합니다.

'(테스트)자동전표_업로드.xlsx'로 엑셀 파일 저장 중...
처리 완료: 60개 행이 '(테스트)자동전표_업로드.xlsx'에 저장됨
엑셀 파일이 성공적으로 생성되었습니다: /Users/haribo/Library/CloudStorage/GoogleDrive-hariqueen98@gmail.com/내 드라이브/업무용/MetaM/auto_ERP/(테스트)자동전표_업로드.xlsx

총 처리 건수: 60건 (차변 59건, 대변 1건)
총 금액: 259,080원
차변 계정: 2개 계정 사용
대변 계정: 미지급금(25300) 1개 계정 사용

관리항목 설정 정보:
- CD_CC (코스트센터): 5020 (고정)
- CD_PARTNER (거래처코드): 101388 (고정)
- CD_PJT (프로젝트코드): 각 팀별 매핑된 코드 

/var/folders/xc/8l9tg1hj3j77dx12_tg131c80000gn/T/ipykernel_3975/3900483466.py:184: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  erp_df[col] = [""] * len(erp_df)
/var/folders/xc/8l9tg1hj3j77dx12_tg131c80000gn/T/ipykernel_3975/3900483466.py:184: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  erp_df[col] = [""] * len(erp_df)
/var/folders/xc/8l9tg1hj3j77dx12_tg131c80000gn/T/ipykernel_3975/3900483466.py:184: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has